# Bank Mass Ranges and GWTC Events Plot

This notebook creates the figure 3 of https://arxiv.org/abs/2507.16022


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gwosc
from gwosc import api

# Add cogwheel to path for plotting utilities
from cogwheel import gw_plotting


In [ ]:
# Hardcoded bank configurations from create_banks.py
bank_configs = {
    3: (3.0, 3.2),
    5: (5.0, 6.0),
    10: (10.0, 12.0),
    20: (20.0, 30.0),
    50: (50.0, 60.0),
    100: (100.0, 300.0),
}

# Create bank names and ranges
bank_names = [f"bank_mchirp_{x}" for x in [3, 5, 10, 20, 50, 100]]
mass_ranges = {
    bank_name: bank_configs[int(bank_name.split("_")[2])]
    for bank_name in bank_names
}

# Create labels for banks
bank_labels = {
    bank_name: r"$\mathcal{M} \in ("
    + f"{mass_ranges[bank_name][0]:.3g},{mass_ranges[bank_name][1]:.3g}"
    + r"){\rm M}_{\odot}$"
    for bank_name in bank_names
}

# Define markers and colors
markers = ["o", "s", "D", "^", "v", "P"]
bank_markers = {
    bank_name: marker for bank_name, marker in zip(bank_names, markers)
}
color_map = plt.colormaps["tab10"]
bank_colors = {bank: color_map(i) for i, bank in enumerate(bank_names)}

In [ ]:
# Fetch GWTC-3-marginal catalog

catalogue = api.fetch_catalog_json("GWTC-4.0")

gwosc_summary = []
for eventname, v in catalogue["events"].items():
    try:
        mchirp = v.get("chirp_mass_source", None)
        mchirp_lower = v.get("chirp_mass_source_lower", None)
        mchirp_upper = v.get("chirp_mass_source_upper", None)
        snr = v.get("network_matched_filter_snr", None)
        redshift = v.get("redshift", None)
        pastro = v.get("p_astro", None)
        catalog_name = v.get("catalog.shortName", None)
        gwosc_summary.append(
            {
                "name": eventname,
                "mchirp": mchirp,
                "mchirp_lower": mchirp_lower,
                "mchirp_upper": mchirp_upper,
                "SNR": snr,
                "p_astro": pastro,
                "redshift": redshift,
                "catalog_name": catalog_name,
            }
        )
    except Exception as e:
        print(f"Skipping Event {eventname} due to error: {e}")

gwosc_summary = pd.DataFrame(gwosc_summary)
print(f"Loaded {len(gwosc_summary)} events from GWTC-3-marginal catalog")
print(f"Events with valid SNR: {(gwosc_summary.SNR > 0).sum()}")


In [ ]:
# Create the banks_mass_ranges_and_gwtc_events.png plot
fig, ax = plt.subplots(figsize=(6, 4))

# Plot GWTC-3-marginal events with SNR weighting
cond = gwosc_summary.SNR > 0
snr_weight = gwosc_summary.loc[cond, "SNR"] / 8
ax.scatter(
    gwosc_summary.loc[cond, "mchirp"] * (1 + gwosc_summary.loc[cond, "redshift"]),
    (gwosc_summary.loc[cond, "mchirp_upper"] - gwosc_summary.loc[cond, "mchirp_lower"])
    * (1 + gwosc_summary.loc[cond, "redshift"])
    * snr_weight,
    s=5,
    marker="o",
    label="GWTC-3-marginal",
    c="black",
    alpha=0.30,
)

ax.set_yscale("log")
ax.set_xscale("log")

# Plot power law scaling
ax.plot(
    np.array([1, 200]),
    (np.array([1, 200]) / 25) ** 1.7 * 10,
    ls="-",
    c="k",
    alpha=1,
    label=r"$\Delta\mathcal{M}\propto \mathcal{M}^{1.7}$",
)

# Plot bank ranges using hardcoded values
for bank_name in bank_names:
    mass_range = mass_ranges[bank_name]
    mass_median = np.exp(0.5 * (np.log(mass_range[0]) + np.log(mass_range[1])))
    mass_width = np.diff(mass_range)[0]
    ax.errorbar(
        x=mass_median,
        y=mass_width,
        xerr=np.array(
            (mass_median - mass_range[0], mass_range[1] - mass_median)
        ).reshape(-1, 1),
        ls="-",
        c=bank_colors[bank_name],
        label=bank_labels[bank_name],
        marker=bank_markers[bank_name],
        ms=0,
        lw=5,
    )

ax.set_xlabel(gw_plotting._LABELS["mchirp"])
ax.set_ylabel(r"$\Delta$" + gw_plotting._LABELS["mchirp"])
leg = ax.legend(bbox_to_anchor=(1.01, 0.99))
ax.grid()
fig.tight_layout()
fig.savefig("figure-3.png", dpi=200)

print("Figure saved as figure-3.png")
